# Properties of the donor during mass transfer
author: [Mathieu Renzo](mailto:mrenzo@flatironinstitute.org)

Study the donor during RLOF

In [ ]:
import sys
# the line below import stuff from git@github.com:mathren90/plotFunc.git
sys.path.append('/mnt/home/mrenzo/codes/python_stuff/plotFunc/')
from MESAreader import *
import matplotlib.pyplot as plt
%matplotlib inline
from plot_defaults import *
# plot_aux defines many auxiliary functions to make plots
from plot_aux import *
from lib_pre_RLOF import *

In [ ]:
set_plot_defaults_from_matplotlibrc('./')

In [ ]:
bin_folder = "/mnt/home/mrenzo/ceph/RESULTS/zeta_oph/zeta_ophiuchi/M1_25_M2_17_P100_Z0.01/re-run_exponential/"
acc_folder = bin_folder+'/binary_to_postRLOF/LOGS2/'
acc_post_RLOF = bin_folder+'/accretor_to_TAMS/LOGS/'
donor_folder = bin_folder+'binary_to_postRLOF/LOGS1/'

In [ ]:
bsrc, bcol = getSrcCol(acc_folder+"/../binary_history.data")
iRLOF = (bsrc[:, bcol.index("rl_relative_overflow_1")] >0)
tRLOF = bsrc[iRLOF, bcol.index("age")]
tRLOF_start = min(tRLOF)
tRLOF_end = max(tRLOF)
src_2, col_2 = getSrcCol(acc_post_RLOF+'/history.data')
tTAMS_acc = max(src_2[:, col_2.index('star_age')])
print(tRLOF_start/1e6, tRLOF_end/1e6)

## binary separation

In [ ]:
fig = plt.figure(figsize=(10,10))
gs = gridspec.GridSpec(120, 100)
ax = fig.add_subplot(gs[:,:])
ax2 = ax.twinx()
# top_ax = fig.add_subplot(gs[:20,:])
# top_ax.set_xticklabels([])


model_number = bsrc[:, bcol.index("model_number")]
iRLOF_start = int(min(model_number[iRLOF]))
separation = bsrc[iRLOF_start, bcol.index("binary_separation")]*Rsun_cm/1e10
rl_1 = bsrc[iRLOF_start, bcol.index("rl_1")]*Rsun_cm/1e10
rl_2 = bsrc[iRLOF_start, bcol.index("rl_2")]*Rsun_cm/1e10
ax.set_xlim(-1.5*rl_1, separation+1.1*rl_2)
# top_ax.set_xlim(ax.get_xlim())
ax.set_xlabel(r"$x \ [10^{10}\, cm]$")
ax.set_ylabel(r"solid lines: $\log_{10}(\rho/\mathrm{[g\ cm^{-3}]})$")
ax2.set_ylabel(r"dashed lines: $\log_{10}(H_p/\mathrm{[g\ cm^{-3}]})$")
#highlight roche lobes
ax.axvspan(-rl_1, rl_1, color="r", alpha=0.2)
ax.text(0, -10.5, "$\mathrm{RL}_\mathrm{donor}$", fontsize=30, transform=ax.transData, ha="center")    

ax.axvspan(separation-rl_2, separation+rl_2, color="b", alpha=0.2)
ax.text(separation, -10.5, "$\mathrm{RL}_\mathrm{accretor}$", fontsize=30, transform=ax.transData, ha="center")    

ax.plot([0, separation], [0.5,0.5], lw=5, c='k', ls='--')
ax.text(1000,0.51, r"$a="+f"{separation:.0f}"+r"\,\times10^{10}\,\mathrm{cm}$", transform=ax.transData, va="bottom", ha="center", fontsize=30)

# plot density profiles

# donor
pfile_donor = donor_folder+'profile38.data'
src, col = getSrcCol(pfile_donor)
log_rho_donor = src[:, col.index("logRho")]
r_donor = src[:, col.index("radius")]*Rsun_cm/1e10
# x_donor = r_donor-rl_1 # 10^10cm units
ax.plot(r_donor, log_rho_donor, c="red")
ax.plot(-r_donor, log_rho_donor, c="red")
H_donor = get_pressure_scale_height(pfile_donor)
ax2.plot(r_donor, np.log10(H_donor), c="red", ls='--')
ax2.plot(-r_donor, np.log10(H_donor), c="red", ls='--')
# ratio = H_donor/max(r_donor)
# top_ax.plot(r_donor, ratio, c='r')
# top_ax.plot(-r_donor, ratio, c='r')


# # accretor
pfile_accretor = acc_folder+'/profile3573.data'
src, col = getSrcCol(pfile_accretor)
log_rho_accretor = src[:, col.index("logRho")]
r_accretor = src[:, col.index("radius")]*Rsun_cm/1e10
ax.plot(r_accretor+separation, log_rho_accretor, c="b")
ax.plot(separation-r_accretor, log_rho_accretor, c="b")
H_accretor = get_pressure_scale_height(pfile_accretor)
ax2.plot(r_accretor+separation, np.log10(H_accretor), c="b", ls='--')
ax2.plot(separation-r_accretor, np.log10(H_accretor), c="b", ls='--')
# ratio = H_accretor/max(r_accretor)
# top_ax.plot(separation+r_accretor, ratio, c='b')
# top_ax.plot(separation-r_accretor, ratio, c='b')

# top_ax.set_ylabel(r"$H_p/R_{\star}$")
# top_ax.set_ylim(0, 1)


From the visualization above, we conclude that:
- the full domain for the RLOF stream is $8\times
  10^{12}\,\mathrm{cm}< x < 2.5\times 10^{13}\,\mathrm{cm}$, this
  includes the surface layers of the donor, the separation, and parts
  of the accretor's envelope
- the pressure scale height is of order $3\times 10^{12}\,\mathrm{cm}$
  at the edge of the domain
- assuming we need 10 mesh points to resolve the scale height, it
  means $\Delta x \simeq 3\times10^{11}\,\mathrm{cm}$ so maybe we
  cannot do all in one piece.

## radius and structure at onset of RLOF

In [ ]:
pfile = donor_folder+'profile38.data'
plot_rho_Hp_mass(pfile, X_onset=1e-4)
plot_rho_Hp_radius(pfile, X_onset=0.42)